In [1]:
!pip install --upgrade nbformat

In [2]:
!pip install prophet

In [3]:
!pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [4]:
import pandas as pd
from prophet import Prophet
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go


In [5]:
df=pd.read_csv('/content/long_data_.csv')

In [6]:
df.head()

,States,Regions,latitude,longitude,Dates,Usage
0,Punjab,NR,31.519974,75.980003,02/01/2019 00:00:00,119.9
1,Haryana,NR,28.450006,77.019991,02/01/2019 00:00:00,130.3
2,Rajasthan,NR,26.449999,74.639981,02/01/2019 00:00:00,234.1
3,Delhi,NR,28.669993,77.230004,02/01/2019 00:00:00,85.8
4,UP,NR,27.599981,78.050006,02/01/2019 00:00:00,313.9


In [7]:
df.shape

(16599, 6)

In [8]:
df['Dates'] = pd.to_datetime(df['Dates'])

In [9]:
df = df.groupby(df['Dates'], as_index = False).mean()
df.head()

<ipython-input-9-8b8d4d91446a>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(df['Dates'], as_index = False).mean()


,Dates,latitude,longitude,Usage
0,2019-01-07,23.17822,81.794533,103.736364
1,2019-01-08,23.17822,81.794533,113.951515
2,2019-01-09,23.17822,81.794533,107.836364
3,2019-01-10,23.17822,81.794533,98.045455
4,2019-01-11,23.17822,81.794533,84.463636


In [10]:
df.shape

(498, 4)

In [11]:
df = df[['Dates','Usage']]

In [12]:
fig = px.line(df, x='Dates', y='Usage')
fig.show()

In [13]:
df.columns = ['ds','y']

In [14]:
df.head()

,ds,y
0,2019-01-07,103.736364
1,2019-01-08,113.951515
2,2019-01-09,107.836364
3,2019-01-10,98.045455
4,2019-01-11,84.463636


In [15]:
model=Prophet()

In [16]:
model.fit(df)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2s3b03fz/iwkj3i16.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2s3b03fz/xqxtwa64.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=11929', 'data', 'file=/tmp/tmp2s3b03fz/iwkj3i16.json', 'init=/tmp/tmp2s3b03fz/xqxtwa64.json', 'output', 'file=/tmp/tmp2s3b03fz/prophet_modelfyzi32z3/prophet_model-20240127073135.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:31:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:31:35 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [17]:
model.component_modes

{'additive': ['weekly',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [18]:
future_dates = model.make_future_dataframe(periods=365,freq='d',include_history=True)
future_dates.shape

(863, 1)

In [19]:
future_dates.head()

,ds
0,2019-01-07
1,2019-01-08
2,2019-01-09
3,2019-01-10
4,2019-01-11


In [20]:
prediction=model.predict(future_dates)

In [21]:
prediction.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2019-01-07,103.380111,90.853800,115.857053,103.380111,103.380111,-0.184882,-0.184882,-0.184882,-0.184882,-0.184882,-0.184882,0.0,0.0,0.0,103.195228
1,2019-01-08,103.377977,90.416684,116.903795,103.377977,103.377977,0.650408,0.650408,0.650408,0.650408,0.650408,0.650408,0.0,0.0,0.0,104.028385
2,2019-01-09,103.375844,90.423719,115.701355,103.375844,103.375844,0.151419,0.151419,0.151419,0.151419,0.151419,0.151419,0.0,0.0,0.0,103.527263
3,2019-01-10,103.373710,90.389978,114.887069,103.373710,103.373710,-0.293240,-0.293240,-0.293240,-0.293240,-0.293240,-0.293240,0.0,0.0,0.0,103.080470
4,2019-01-11,103.371577,89.957285,115.621058,103.371577,103.371577,-0.470830,-0.470830,-0.470830,-0.470830,-0.470830,-0.470830,0.0,0.0,0.0,102.900746


In [22]:
trace_open = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat"],
    mode = 'lines',
    name="Forecast"
)
trace_high = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_upper"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Higher uncertainty interval"
)
trace_low = go.Scatter(
    x = prediction["ds"],
    y = prediction["yhat_lower"],
    mode = 'lines',
    fill = "tonexty",
    line = {"color": "#57b8ff"},
    name="Lower uncertainty interval"
)
trace_close = go.Scatter(
    x = df["ds"],
    y = df["y"],
    name="Data values"
)

#make list for all three scattle objects.
data = [trace_open,trace_high,trace_low,trace_close]
# Construct a new Layout object.
#title - It will display string as a title of graph
layout = go.Layout(title="Power consumption forecasting")
#A list or tuple of trace instances (e.g. [Scatter(…), Bar(…)]) or A single trace instance (e.g. Scatter(…), Bar(…), etc.)
#A list or tuple of dicts of string/value properties where: - The ‘type’ property specifies the trace type.

fig = go.Figure(data=data)
fig.show()

In [23]:

fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])
#You can add traces using an Express plot by using add_trace
fig.add_trace(go.Scatter(x=prediction['ds'], y=prediction['yhat'],
                   mode='lines+markers',
                    name='predicted'))
#To display a figure using the renderers framework, you call the .show() method on a graph object figure, or pass the figure to the plotly.io.show function.
#With either approach, plotly.py will display the figure using the current default renderer(s).
fig.show()

In [24]:
#Return a Numpy representation of the DataFrame.
y_true = df['y'].values

#Here we have specified [:498] because in y_true we have 498 data points so for comparing both series we need equal shape of series.
y_pred = prediction['yhat'][:498].values

#Parameters:
#y_truearray-like of shape = (n_samples) or (n_samples, n_outputs)
#Ground truth (correct) target values.

#y_predarray-like of shape = (n_samples) or (n_samples, n_outputs)
#Estimated target values.

mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

MAE: 7.910


# **Optimizing the model for better forecasting**

In [25]:
model1=Prophet(daily_seasonality=True).add_seasonality(name='yearly',period=365,fourier_order=70)

In [26]:
model1.fit(df)

INFO:prophet:Found custom seasonality named 'yearly', disabling built-in 'yearly' seasonality.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2s3b03fz/u7_1_t3z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp2s3b03fz/nmx5kj2j.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83566', 'data', 'file=/tmp/tmp2s3b03fz/u7_1_t3z.json', 'init=/tmp/tmp2s3b03fz/nmx5kj2j.json', 'output', 'file=/tmp/tmp2s3b03fz/prophet_modelbdfepxho/prophet_model-20240127073156.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:31:56 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:31:56 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [27]:
model1.component_modes

{'additive': ['yearly',
  'weekly',
  'daily',
  'additive_terms',
  'extra_regressors_additive',
  'holidays'],
 'multiplicative': ['multiplicative_terms', 'extra_regressors_multiplicative']}

In [28]:
future_dates1=model1.make_future_dataframe(periods=365)

In [29]:
prediction1=model1.predict(future_dates1)

In [30]:
from sklearn.metrics import mean_absolute_error
y_true = df['y'].values
y_pred = prediction1['yhat'][:498].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

MAE: 5.600


In [31]:
import plotly.graph_objects as go
fig = go.Figure([go.Scatter(x=df['ds'], y=df['y'],mode='lines',
                    name='Actual')])

fig.add_trace(go.Scatter(x=prediction1['ds'], y=prediction1['yhat'],
                   mode='lines+markers',
                    name='predicted'))

fig.show()